# 2. Feature engineering y entrenamiento del modelo de detección LightGBM

Este notebook implementa un pipeline de entrenamiento robusto para la detección de fugas de agua. Debido al gran volumen de datos ($>75$ millones de registros) y las limitaciones de memoria RAM, se ha diseñado una estrategia de carga distribuida con Dask y muestreo inteligente.

El objetivo es entrenar tres modelos de Gradient Boosting (LightGBM) independientes para predecir la probabilidad de fuga en tres horizontes temporales: Hoy (1h), Mañana (24h) y 7 Días (168h). Estos modelos alimentarán posteriormente el sistema de Meta-Análisis para la toma de decisiones.

## 2.1. Configuaración inicial

Definimos las rutas de los datos procesados y los parámetros globales del entrenamiento. Se establece una ventana de tiempo continua (Enero-Marzo) para garantizar la coherencia de las series temporales, y un porcentaje de subsampling para asegurar que el dataset quepa en la memoria RAM disponible durante el entrenamiento.

In [1]:
# Importamos librerías
import pandas as pd
import numpy as np
import lightgbm as lgb
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from tqdm import tqdm
import dask.dataframe as dd

In [2]:
# Congiguración del Proyecto
FINAL_DATASET_PATH = '../data/processed-data/dataset_FINAL_COMPLETO/'
ID_COLUMN = 'POLISSA_SUBM'

# Definición de los horizontes de predicción (en número de registros/horas)
HORIZONTES_PREDICCION = {'HOY': 1, 'MANANA': 24, '7DIAS': 168}

# Configuración de fechas de entrenamiento
TRAIN_START_DATE = '2024-01-01'
TRAIN_END_DATE = '2024-06-30' 

# Usamos el 25% de los datos de ese rango de fechas para evitar ArrowMemoryError
SUBSAMPLE_PERCENT = 0.25

## 2.2. Selección de características

Definimos explícitamente qué columnas son **Metadata** (no predictivas) y cuáles son las **Features Base** que alimentarán al modelo.
Esta selección optimizada es crucial para reducir el consumo de memoria.

Se realiza una selección explícita de variables (Feature Selection).

* Se excluyen metadatos administrativos (IDs, fechas de facturación) y columnas redundantes para evitar data leakage y reducir la dimensionalidad.

* Se seleccionan 32 variables clave agrupadas en: Consumo, Clima, Entorno Socioeconómico, Demografía e Infraestructura. Esta selección manual optimiza el rendimiento del modelo LightGBM y reduce el riesgo de sobreajuste.

In [3]:
# Lista de columnas de metadatos (para exclusión)
METADATA_AND_TARGET_COLUMNS = [
    'POLISSA_SUBM', 'NUMEROSERIECONTADOR', 'SECCIO_CENSAL', 'KEY_DISTRITO', 'KEY_SECCION', 
    'FECHA_HORA', 'FECHA', 'HORA', 'DATA_INI_FACT', 'DATA_FIN_FACT', 'FECHA_HORA_CRONO',
    'FUGA_DETECTADA', 'FUGA_REITERADA', 
]

# Feature base (Numéricas + Categóricas)
FEATURE_COLUMNS_BASE = [
    'CONSUMO_REAL', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 
    'HUMEDAD_RELATIVA_MEDIA', 'FESTIVO', 'Renda_Media_Euros', 'Antig_1901_a_1940', 
    'Antig_1941_a_1950', 'Antig_1951_a_1960', 'Antig_1961_a_1970', 'Antig_1971_a_1980', 
    'Antig_1981_a_1990', 'Antig_1991_a_2000', 'Antig_2001_a_2010', 'Antig_2011_a_2020', 
    'Antig_2021_a_2030', 'Antig_Menor_1901', 'Pob_0_14_anys', 'Pob_15_24_anys', 
    'Pob_25_39_anys', 'Pob_40_64_anys', 'Pob_65_o_mas', 'Pob_Total_Seccio', 
    'Pct_Pob_0_14_anys', 'Pct_Pob_15_24_anys', 'Pct_Pob_25_39_anys', 
    'Pct_Pob_40_64_anys', 'Pct_Pob_65_o_mas', 'Num_Obres_Recents',
    'US_AIGUA_SUBM', 'TIPO_DIA' 
]

## 2.3. Carga de datos distribuída (Solución Anti-Memoria)

Utilizamos **Dask** para cargar el dataset masivo y realizar un muestreo aleatorio (`.sample()`) antes de cargarlo en la memoria RAM. Esto evita el error `ArrowMemoryError` al no intentar leer los 16GB de golpe.

Para superar el ArrowMemoryError causado por el tamaño del dataset completo ($16 \text{ GiB}$), implementamos una estrategia de carga en dos fases:
* Filtrado Distribuido (Lazy): Usamos Dask para filtrar por rango de fechas y aplicar un muestreo aleatorio (sample) sin cargar los datos en memoria.

* Consolidación (Compute): Solo traemos a la memoria RAM de Pandas el subconjunto resultante, garantizando que el entorno de trabajo se mantenga estable.

In [4]:
print("--- 2. Carga Distribuida y Muestreo ---")

try:
    # 1. Cargar puntero Dask (Lazy load)
    df_dask_full = dd.read_parquet(FINAL_DATASET_PATH)
    
    # 2. Asegurar formato de fecha
    df_dask_full['FECHA'] = dd.to_datetime(df_dask_full['FECHA'])
    
    # 3. APLICAR FILTRO DE FECHAS (Paso 1: Seleccionar el periodo correcto)
    print(f"Filtrando periodo: {TRAIN_START_DATE} a {TRAIN_END_DATE}...")
    df_dask_window = df_dask_full[
        (df_dask_full['FECHA'] >= TRAIN_START_DATE) & 
        (df_dask_full['FECHA'] <= TRAIN_END_DATE)
    ]
    
    # 4. APLICAR SUBSAMPLING (Paso 2: Reducir volumen para la RAM)
    if SUBSAMPLE_PERCENT < 1.0:
        print(f"Aplicando muestreo del {SUBSAMPLE_PERCENT:.0%} sobre el periodo seleccionado...")
        # .sample() en Dask es distribuido y eficiente
        df_dask_final = df_dask_window.sample(frac=SUBSAMPLE_PERCENT, random_state=42)
    else:
        df_dask_final = df_dask_window
    
    # 5. CONSOLIDACIÓN EN RAM (Ahora sí es seguro hacer .compute())
    df = df_dask_final.compute()
    
    print(f"✅ Dataset cargado en RAM: {df.shape[0]} filas.")

except Exception as e:
    print(f"ERROR CRÍTICO: {e}")
    exit()

# 6. Ordenamiento cronológico final (Vital para los Lags)
print("Ordenando datos...")
df['FECHA_HORA_CRONO'] = pd.to_datetime(df['FECHA'].astype(str) + ' ' + df['HORA'].astype(str), errors='coerce')
df = df.sort_values(by=[ID_COLUMN, 'FECHA_HORA_CRONO']).reset_index(drop=True)

--- 2. Carga Distribuida y Muestreo ---
Filtrando periodo: 2024-01-01 a 2024-06-30...
Aplicando muestreo del 25% sobre el periodo seleccionado...
✅ Dataset cargado en RAM: 9238937 filas.
Ordenando datos...


## 2.4. Feature Engineering

Dado que LightGBM no es un modelo secuencial (como LSTM), debemos crear explícitamente características que capturen la temporalidad y la tendencia:

* Lags (Retardos): Variables que indican el consumo y clima de hace 1h, 6h, 12h, 24h y 72h.

* Rolling Windows (Medias Móviles): Cálculo de la media y desviación estándar del consumo en los últimos 7 días para establecer una "línea base" de comportamiento normal.

* Ratios de Desviación: Nuevas variables ingenieriles (RATIO_CONSUMO_MEDIA) que cuantifican cuánto se desvía el consumo actual respecto a su media histórica, facilitando la detección de anomalías de volumen.

* Targets: Generación de las variables objetivo desplazadas hacia el futuro para los tres horizontes de predicción.

In [5]:
# Definición de Features (Ingeniería de Características)
LAG_FEATURES = ['CONSUMO_REAL', 'TEMP_MEDIA', 'PRECIPITACION']
LAG_STEPS = [1, 6, 12, 24, 72] 

for col in tqdm(LAG_FEATURES, desc="Creando Lags"):
    for lag in LAG_STEPS:
        # Al estar ordenado por Cliente+Fecha, el shift funciona correctamente
        df[f'{col}_LAG_{lag}H'] = df.groupby(ID_COLUMN)[col].shift(lag)

# Rolling (Ventana móvil de 7 días)
WINDOW_SIZE = 168
df['CONSUMO_ROLLING_MEAN_7D'] = df.groupby(ID_COLUMN)['CONSUMO_REAL'].transform(lambda x: x.rolling(WINDOW_SIZE, min_periods=1).mean())
df['CONSUMO_ROLLING_STD_7D'] = df.groupby(ID_COLUMN)['CONSUMO_REAL'].transform(lambda x: x.rolling(WINDOW_SIZE, min_periods=1).std())


# --- *** MEJORA: RATIOS DE DESVIACIÓN (NUEVO) *** ---
# Estas features le gritan al modelo: "¡Oye, esto es 5 veces más alto de lo normal!"
epsilon = 0.001 # Para evitar división por cero
df['RATIO_CONSUMO_MEDIA_7D'] = df['CONSUMO_REAL'] / (df['CONSUMO_ROLLING_MEAN_7D'] + epsilon)
df['DIFF_CONSUMO_MEDIA_7D'] = df['CONSUMO_REAL'] - df['CONSUMO_ROLLING_MEAN_7D']


# Targets (Desplazamos hacia atrás para ver el futuro)
TARGET_COLS = ['TARGET_HOY', 'TARGET_MANANA', 'TARGET_7DIAS']
df['TARGET_HOY'] = df['FUGA_DETECTADA'].shift(-1)
df['TARGET_MANANA'] = df['FUGA_DETECTADA'].shift(-24)
df['TARGET_7DIAS'] = df['FUGA_DETECTADA'].shift(-168)

Creando Lags: 100%|██████████| 3/3 [00:06<00:00,  2.24s/it]


## 2.5. Preparación para el set de entrenamiento

Preparamos las matrices finales para el modelo:

* Limpieza de Nulos: Eliminamos las filas iniciales y finales que contienen NaN debido a la generación de Lags y Targets.

* Definición de Tipos: Convertimos explícitamente las variables categóricas (TIPO_DIA, US_AIGUA_SUBM) al tipo category de Pandas para que LightGBM las procese de forma óptima.

* Time-Split: Dividimos los datos cronológicamente (80% Pasado para Entrenar, 20% Futuro para Test) para simular un escenario de producción real y evitar mirar al futuro (look-ahead bias).

In [6]:
# --- 4. Definición de Sets y Formateo Final ---
print("\n--- 4. Definición de Sets y Formateo ---")

# 4.1. Definición de X_COLS (Lógica de Sustracción - MÁS SEGURA)
# En lugar de listar qué queremos, listamos qué NO queremos.
# Así, las nuevas features (Ratios, Diffs) entran automáticamente.

COLS_TO_EXCLUDE = METADATA_AND_TARGET_COLUMNS + TARGET_COLS + ['FECHA_HORA_CRONO']

# X_COLS = Todas las columnas presentes MENOS las excluidas
X_COLS = [col for col in df.columns if col not in COLS_TO_EXCLUDE]

print(f"Features finales para el entrenamiento (X): {len(X_COLS)} columnas.")
print(f"Ejemplo de features incluidas: {X_COLS[-5:]}") # Verificamos que estén las nuevas

# 4.2. Eliminamos NaNs resultantes del lagging y shifting
# Usamos el target más lejano y las nuevas features como referencia
df = df.dropna(subset=['TARGET_7DIAS'] + X_COLS) 

# 4.3. Definición de Features Categóricas (para LightGBM)
CATEGORICAL_COLS = ['US_AIGUA_SUBM', 'TIPO_DIA'] 
for col in CATEGORICAL_COLS:
    if col in X_COLS:
        df[col] = df[col].astype('category')

# 4.4. Aplicando Time-Split Cronológico (80% Train, 20% Test)
TEST_SIZE_PERCENTAGE = 0.20
split_index = int(len(df) * (1 - TEST_SIZE_PERCENTAGE))

df_train = df.iloc[:split_index].copy()
df_test = df.iloc[split_index:].copy()

print(f"Set de Entrenamiento (Pasado): {df_train.shape[0]} filas")
print(f"Set de Prueba (Futuro/Simulación): {df_test.shape[0]} filas")



--- 4. Definición de Sets y Formateo ---
Features finales para el entrenamiento (X): 52 columnas.
Ejemplo de features incluidas: ['PRECIPITACION_LAG_72H', 'CONSUMO_ROLLING_MEAN_7D', 'CONSUMO_ROLLING_STD_7D', 'RATIO_CONSUMO_MEDIA_7D', 'DIFF_CONSUMO_MEDIA_7D']
Set de Entrenamiento (Pasado): 7245110 filas
Set de Prueba (Futuro/Simulación): 1811278 filas


## 2.6. Entrenamiento del modelo LightGBM

Entrenamos los tres modelos (Hoy, Mañana, 7 Días) utilizando un bucle optimizado para estabilidad en Windows:

* Estabilidad: Se fuerza n_jobs=1 y force_col_wise=True para evitar conflictos de memoria y errores de acceso (Access Violation).

* Robustez: Se realiza una limpieza final de tipos numéricos (eliminación de comas decimales) justo antes del entrenamiento.

* Validación: Se utiliza un conjunto de validación interno (15% del train) con Early Stopping para detener el entrenamiento automáticamente cuando el modelo deja de aprender, preveniendo el sobreajuste.

* Persistencia: Los modelos resultantes se guardan en formato .joblib para su despliegue en la aplicación.

In [7]:
# 2.6. Entrenamiento del modelo LightGBM (Versión Estable Windows)
import gc

print("---"*20)
print("\nENTRENAMIENTO DEL MODELO LIGHTGBM (MODO SEGURO)\n")
print("---"*20)

modelos_finales = {}
output_dir = '../data/processed-data/'
os.makedirs(output_dir, exist_ok=True)

# --- 1. PREPARACIÓN ÚNICA DE LA MATRIZ X ---
print("Preparando matriz X única...")
X_base = df_train[X_COLS].copy()

# Limpieza de tipos (Una sola vez)
num_cols = [c for c in X_COLS if c not in CATEGORICAL_COLS]
for col in num_cols:
    if X_base[col].dtype == 'object':
        X_base[col] = X_base[col].astype(str).str.replace(',', '.', regex=False)
    X_base[col] = pd.to_numeric(X_base[col], errors='coerce').fillna(0.0)

for col in CATEGORICAL_COLS:
    if col in X_base.columns:
        X_base[col] = X_base[col].astype(str).astype('category')

print(f"Matriz X preparada: {X_base.shape}")

# --- 2. BUCLE DE ENTRENAMIENTO ---
for target_col in TARGET_COLS:
    print(f"\n[PROCESANDO] {target_col}...")
    
    y = df_train[target_col].astype(int)

    # División interna
    X_fit, X_val, y_fit, y_val = train_test_split(
        X_base, y, test_size=0.15, random_state=42, stratify=y
    )
    
    gc.collect()
    
    # --- CONFIGURACIÓN (Corrección de Estabilidad) ---
    lgbm = lgb.LGBMClassifier(
        objective='binary',
        metric='auc', 
        is_unbalance=True,
        n_estimators=300,
        learning_rate=0.05,
        n_jobs=1,  # <--- CRÍTICO: Usar 1 núcleo evita el Access Violation en Windows
        random_state=42,
        verbose=-1,
        force_col_wise=True
    )
    
    # --- ENTRENAMIENTO (Con copias de seguridad) ---
    lgbm.fit(
        X_fit.copy(), y_fit.copy(), # Copias explícitas
        categorical_feature=[c for c in X_COLS if c in CATEGORICAL_COLS],
        eval_set=[(X_val.copy(), y_val.copy())], # Copias explícitas
        callbacks=[lgb.early_stopping(50, verbose=True)]
    )
    
    modelos_finales[target_col] = lgbm
    joblib.dump(lgbm, os.path.join(output_dir, f'lgbm_model_{target_col}.joblib'))
    
    del X_fit, X_val, y_fit, y_val, lgbm
    gc.collect()

print("\n✅ Todos los modelos han sido entrenados y guardados.")

------------------------------------------------------------

ENTRENAMIENTO DEL MODELO LIGHTGBM (MODO SEGURO)

------------------------------------------------------------
Preparando matriz X única...
Matriz X preparada: (7245110, 52)

[PROCESANDO] TARGET_HOY...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's auc: 0.982559

[PROCESANDO] TARGET_MANANA...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's auc: 0.980912

[PROCESANDO] TARGET_7DIAS...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[300]	valid_0's auc: 0.971041

✅ Todos los modelos han sido entrenados y guardados.


## 2.7. Evaluación final del modelo

Evaluamos el rendimiento de los modelos sobre el conjunto de prueba (datos futuros no vistos). Además de las métricas estándar (AUC-ROC, AUC-PR), implementamos una Búsqueda de Umbral Óptimo (Threshold Tuning).

Analizamos cómo varía la Precisión y el Recall al cambiar el umbral de decisión (0.3, 0.4, 0.5, 0.6) para encontrar el punto de equilibrio que maximice la detección de fugas reales (F1-Score) sin disparar las falsas alarmas.

In [8]:
# Evaluación Final en el Test Set
print("---"*20)
print("\nEVALUACIÓN FINAL (TEST SET) \n")
print("---"*20)

# Preparamos X_test (Limpieza de tipos igual que en train)
X_test_final = df_test[X_COLS].copy()
num_cols = [c for c in X_COLS if c not in CATEGORICAL_COLS]
for col in num_cols:
    if X_test_final[col].dtype == 'object':
        X_test_final[col] = X_test_final[col].astype(str).str.replace(',', '.', regex=False)
    X_test_final[col] = pd.to_numeric(X_test_final[col], errors='coerce').fillna(0.0)

# DataFrame para guardar resultados
df_resultados = df_test[[ID_COLUMN, 'FECHA_HORA_CRONO']].copy()

for target_col, modelo in modelos_finales.items():
    y_test_final = df_test[target_col].astype(int)
    
    # 1. Predicción de Probabilidad
    y_pred_proba = modelo.predict_proba(X_test_final, raw_score=False)[:, 1]
    
    # Guardar en CSV
    df_resultados[f'PROB_{target_col}'] = y_pred_proba
    df_resultados[f'REAL_{target_col}'] = y_test_final.values

    # 2. Métricas Globales
    auc_pr = average_precision_score(y_test_final, y_pred_proba)
    roc_auc = roc_auc_score(y_test_final, y_pred_proba)
    
    print("="*60)
    print(f"RESULTADOS PARA: {target_col}")
    print(f"AUC-PR: {auc_pr:.4f} | AUC-ROC: {roc_auc:.4f}")
    print("-" * 60)
    
    # --- *** ESTRATEGIA: BÚSQUEDA DE UMBRAL ÓPTIMO *** ---
    # Probamos umbrales más bajos para cazar más fugas (subir Recall)
    thresholds = [0.3, 0.4, 0.5, 0.6]
    
    print(f"{'Umbral':<10} {'Precision':<15} {'Recall':<15} {'F1-Score':<10}")
    
    best_f1 = 0
    best_thresh = 0.5
    
    for thresh in thresholds:
        y_pred_custom = (y_pred_proba > thresh).astype(int)
        report = classification_report(y_test_final, y_pred_custom, output_dict=True)
        
        # Métricas de la clase '1' (Fuga)
        prec = report['1']['precision']
        rec = report['1']['recall']
        f1 = report['1']['f1-score']
        
        print(f"{thresh:<10} {prec:<15.4f} {rec:<15.4f} {f1:<10.4f}")
        
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
            
    print("-" * 60)
    print(f"✅ RECOMENDACIÓN: Usar umbral > {best_thresh} para este horizonte.")

# Exportar CSV
csv_path = '../data/analisis_predicciones.csv'
df_resultados.to_csv(csv_path, index=False, sep=';', decimal=',')
print(f"\n✅ Archivo de análisis guardado: {csv_path}")
print("✅ ENTRENAMIENTO COMPLETADO.")

------------------------------------------------------------

EVALUACIÓN FINAL (TEST SET) 

------------------------------------------------------------
RESULTADOS PARA: TARGET_HOY
AUC-PR: 0.8685 | AUC-ROC: 0.7288
------------------------------------------------------------
Umbral     Precision       Recall          F1-Score  
0.3        0.8241          0.6987          0.7562    
0.4        0.8333          0.6152          0.7078    
0.5        0.8464          0.5354          0.6559    
0.6        0.8617          0.4728          0.6106    
------------------------------------------------------------
✅ RECOMENDACIÓN: Usar umbral > 0.3 para este horizonte.
RESULTADOS PARA: TARGET_MANANA
AUC-PR: 0.8578 | AUC-ROC: 0.7126
------------------------------------------------------------
Umbral     Precision       Recall          F1-Score  
0.3        0.8186          0.6573          0.7291    
0.4        0.8314          0.5994          0.6966    
0.5        0.8386          0.5206          0.6424  

## 2.8 Exportación de los resultados a CSV para test de Meta-Análisis

Exportamos las probabilidades predichas y los valores reales a un archivo CSV (analisis_predicciones.csv). Este archivo será el input para el siguiente notebook (Meta-Análisis), donde aplicaremos la lógica de negocio y las reglas de tendencias (deltas) sin necesidad de re-ejecutar los modelos.

In [ ]:
# Exportar CSV para Meta-Análisis
csv_path = '../data/processed-data/analisis_predicciones.csv'
df_resultados.to_csv(csv_path, index=False, sep=';', decimal=',')